## API들을 활용한 통합 workflow 구현

In [1]:
import os, sys, requests
from pathlib import Path
from dotenv import load_dotenv
from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler

In [2]:
load_dotenv('../.env')
SLACK_BOT_TOKEN = os.getenv("SLACK-BOT")
CHANNEL_ID = 'C086KU2133M' # Channel ID
SLACK_APP_TOKEN = os.getenv("SLACK-APP")
OPEN_AI_KEY = os.getenv("OPEN-AI")

In [3]:
sys.path.append('../python')
from utils import download_file, upload_to_s3, is_pdf_by_signature, encode_image, pdf_to_image, search_with_google_api, add_receipt_to_notion_no_assistant
from chains import OCRChain, SearchChain, CategoryAssistantChain
from prompts import analysis_prompt
from tools import search_tool
from langchain.chains import SequentialChain, LLMChain
from langchain.chat_models import ChatOpenAI

In [4]:
# Slack 앱을 초기화합니다
app = App(token=SLACK_BOT_TOKEN)

In [5]:
current_path = Path.cwd()

### 이벤트 처리

In [6]:
# 모든 이벤트를 로깅하는 미들웨어
@app.middleware
def log_request(logger, body, next):
    print(f"이벤트 수신: {body['event']['type']}")
    next()
    
# 파일이 공유될 때 실행되는 이벤트 리스너
@app.event("file_shared")
def handle_file_shared(event, say, logger, client):
    # 이미지 파일인지 확인합니다
    file_id = event["file_id"]
    logger.info(f"새로운 이미지가 공유되었습니다. 파일 ID: {file_id}")
    # 파일 정보를 가져옵니다
    file_info = client.files_info(file=file_id)
    file = file_info["file"]
    file_url = file["url_private_download"]
    image_path = current_path / '../image' / file["name"]
    if download_file(file_url, image_path, SLACK_BOT_TOKEN):
        say(f"파일 '{image_path}'을 성공적으로 다운로드했습니다.")
        #####################################
        # langchain으로 정보 처리
        # OCRChain 초기화
        ocr_chain = OCRChain()
        # SearchChain 초기화
        search_chain = SearchChain(search_tool)
        # Analysis Chain (검색결과로부터 업종 판단) 초기화
        analysis_chain = LLMChain(
            llm = ChatOpenAI(
                model="gpt-4", 
                temperature=0,
                openai_api_key = OPEN_AI_KEY
            ),
            prompt=analysis_prompt,
            output_key="business_category",  # 출력 키를 명시적으로 설정
        )
        sequential_chain = SequentialChain(
            chains=[ocr_chain, search_chain, analysis_chain],
            input_variables=["image_path"],
            output_variables=["business_category", "ocr_response"],
            verbose=True
        )
        # 실행
        result = sequential_chain.invoke({"image_path": image_path})
        print("\n최종 결과:")
        print(result["business_category"])
        
        # Notion DB에 정보 추가
        try:
            s3_file_url = upload_to_s3(image_path, image_path.name)
            add_receipt_to_notion_no_assistant(result['ocr_response'], s3_file_url, result['business_category'])
            app.client.chat_postMessage(channel=CHANNEL_ID, text= f"데이터가 성공적으로 Notion에 저장되었습니다.")
            app.client.chat_postMessage(channel=CHANNEL_ID, text= result['assistant_response'])
        except Exception as e:
            logger.error(f"Notion에 데이터를 추가하는 중 오류 발생: {e}")
        
# message에 대한 처리는 필요 없음
@app.event("message")
def handle_message_events(event, logger, say):
    pass

# 에러 핸들러
@app.error
def custom_error_handler(error, body, logger):
    print(f"에러 발생: {error}")

In [7]:
handler = SocketModeHandler(app, SLACK_APP_TOKEN)
handler.start()

⚡️ Bolt app is running!
이벤트 수신: message
이벤트 수신: message
이벤트 수신: file_shared


/var/folders/39/jrqp49_54fj1tl_h3pjv45780000gn/T/ipykernel_20631/3722578529.py:28: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/var/folders/39/jrqp49_54fj1tl_h3pjv45780000gn/T/ipykernel_20631/3722578529.py:27: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  analysis_chain = LLMChain(




> Entering new SequentialChain chain...
raw_response = {'상호명': '캐롯손해보험', '날짜': '2023-12-02', '항목': [], '총액': 28120}
business_name =  캐롯손해보험
item =  
query =  캐롯손해보험

> Finished chain.

최종 결과:
"손해보험"
데이터가 성공적으로 Notion에 저장되었습니다: 17d81589-428d-81bb-8e7a-ffd1b98b3179


Notion에 데이터를 추가하는 중 오류 발생: The request to the Slack API failed. (url: https://slack.com/api/chat.postMessage)
The server responded with: {'ok': False, 'error': 'not_in_channel'}


이벤트 수신: message


KeyboardInterrupt: 

on_error invoked (session id: ae073ff5-6e69-42be-bc33-eb02582ce0df, error: ConnectionResetError, message: [Errno 54] Connection reset by peer)
on_error invoked (session id: b7ef455a-f494-45cb-af2f-ab252d16170a, error: ConnectionResetError, message: [Errno 54] Connection reset by peer)
on_error invoked (session id: 07624ebb-bb9c-4a8b-b75e-0ba40c6a762b, error: ConnectionResetError, message: [Errno 54] Connection reset by peer)
on_error invoked (session id: 183c3cc6-bcd2-4ef6-95af-69a6f6ed5255, error: ConnectionResetError, message: [Errno 54] Connection reset by peer)
on_error invoked (session id: 399e6d77-5cd1-47e5-b43b-864ab4603d40, error: ConnectionResetError, message: [Errno 54] Connection reset by peer)
on_error invoked (session id: e6e05239-cdba-4d78-8c91-23d861ee10b5, error: ConnectionResetError, message: [Errno 54] Connection reset by peer)
on_error invoked (session id: 80d24f1b-796a-4e95-81b2-7f253a5ae1a8, error: ConnectionResetError, message: [Errno 54] Connection reset by peer)

이벤트 수신: message


Failed to send a request to Slack API server: <urlopen error [Errno 8] nodename nor servname provided, or not known>
Failed to send a request to Slack API server: <urlopen error [Errno 8] nodename nor servname provided, or not known>
Failed to check the current session or reconnect to the server (session id: 2ea67e4b-8a01-4560-ac19-13e5a9f83131, error: URLError, message: <urlopen error [Errno 8] nodename nor servname provided, or not known>)
Failed to send a request to Slack API server: <urlopen error [Errno 8] nodename nor servname provided, or not known>
Failed to send a request to Slack API server: <urlopen error [Errno 8] nodename nor servname provided, or not known>
Failed to check the current session or reconnect to the server (session id: 2ea67e4b-8a01-4560-ac19-13e5a9f83131, error: URLError, message: <urlopen error [Errno 8] nodename nor servname provided, or not known>)
Failed to send a request to Slack API server: <urlopen error [Errno 8] nodename nor servname provided, or no

이벤트 수신: message
